![e-stude.com Treinamentos em Computação](files/logo_e-stude.png)

<h1 align="center"> Introdução ao Processamento de Linguagem Natural (PLN) Usando Python </h1>
<h3 align="center"> Professor Fernando Vieira da Silva MSc.</h3>

<h2> Técnicas para Pré-Processamento - Parte 2</h2>

<p>Uma vez que o texto já foi devidamente tratado, removendo stopwords e pontuações, e aplicando stemming ou lemmatization, agora precisamos contar a frequência das palavras (ou n-grams) que utilizaremos em seguida como atributos para as técnicas de aprendizado de máquina.</p>

<b>1. TF-IDF (Term Frequency - Inverse Document Frequency)</b>

<p><b>Term Frequency:</b> um termo que aparece muito em um documento, tende a ser um termo importante. Em resumo, divide-se o número de vezes em que um termo apareceu pelo maior número de vezes em que algum outro termo apareceu no documento.</p>

tf<sub>wd</sub> = f<sub>wd</sub> / m<sub>wd</sub>

onde:<br>
f<sub>wd</sub> é o número de vezes em que o termo <i>w</i> aparece no documento <i>d</i>.<br>
m<sub>wd</sub> é o maior valor de f<sub>wd</sub> obtido para algum termo do documento <i>d</i><br>

<p><b>Inverse Document Frequency:</b> um termo que aparece em poucos documentos pode ser um bom descriminante. Obtem-se dividindo o número de documentos pelo número de documentos em que o termo aparece.</p>

idf<sub>w</sub> = log<sub>2</sub>(n / n<sub>w</sub>)

onde:<br>
n é o número de documentos no corpus
n<sub>w</sub> é o número de documentos em que o termo <i>w</i> aparece.

Na prática, usa-se:

tf-idf = tf<sub>wd</sub> * idf<sub>w</sub>

Podemos calcular o TF-IDF de um corpus usando o pacote <b>scikit-learn</b>. Primeiramente, vamos abrir novamente o texto de Hamlet e armazenar as sentenças em uma ndarray do numpy (como se cada sentença fosse um documento do corpus):

In [9]:
import nltk
import numpy as np
from nltk.tokenize import sent_tokenize

hamlet_raw = nltk.corpus.gutenberg.raw('shakespeare-hamlet.txt')

sents = sent_tokenize(hamlet_raw)

hamlet_np = np.array(sents)

print(hamlet_np.shape)


(2355,)


<p>Agora vamos definir uma função para tokenização pelo scikit-learn.</p>

In [24]:
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import string
from nltk.corpus import wordnet

stopwords_list = stopwords.words('english')

lemmatizer = WordNetLemmatizer()

def my_tokenizer(doc):
    words = word_tokenize(doc)
    
    pos_tags = pos_tag(words)
    
    non_stopwords = [w for w in pos_tags if not w[0].lower() in stopwords_list]
    
    non_punctuation = [w for w in non_stopwords if not w[0] in string.punctuation]
    
    lemmas = []
    for w in non_punctuation:
        if w[1].startswith('J'):
            pos = wordnet.ADJ
        elif w[1].startswith('V'):
            pos = wordnet.VERB
        elif w[1].startswith('N'):
            pos = wordnet.NOUN
        elif w[1].startswith('R'):
            pos = wordnet.ADV
        else:
            pos = wordnet.NOUN
        
        lemmas.append(lemmatizer.lemmatize(w[0], pos))

    return lemmas
    
    

E essa função será chamada pelo objeto TfidfVectorizer

In [25]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer(tokenizer=my_tokenizer)

tfs = tfidf_vectorizer.fit_transform(hamlet_np)

print(tfs.shape)

(2355, 4305)


In [28]:
print([k for k in tfidf_vectorizer.vocabulary_.keys()][:20])

['roast', 'spred', 'stoupe', 'play', 'cheff', 'vpo', 'amaz', 'beggers', 'opprest', 'rag', 'staires', 'whipt', 'turneth', 'foreknow', 'auouch', 'germaine', 'rend', 'dishonour', 'bleede', 'among']


<b>2. TF-IDF de N-gramas</b>

Opcionalmente, podemos obter os atributos tf-idf de n-grams, combinando as classes CountVectorizer e TfidfTransformer. Em nosso exemplo, vamos utilizar apenas trigramas:

In [30]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

count_vect = CountVectorizer(ngram_range=(3,3))

n_gram_counts = count_vect.fit_transform(hamlet_np)

tfidf_transformer = TfidfTransformer()

tfs_ngrams = tfidf_transformer.fit_transform(n_gram_counts)

print(tfs_ngrams.shape)

(2355, 23483)


<b>3. Redução de Dimensionalidade</b>

<p>A transformação do corpus em atributos contendo as frequências TF-IDF em geral resultará numa ndarray bastante esparsa, ou seja, com muitas dimensões. Porém, além de isso tornar o treinamento de algoritmos mais demorado e custoso (computacionalmente falando), muitas dessas dimensões provavelmente são pouco representativas ou mesmo podem causar ruído durante o treinamento. Para resolver esse problema, podemos aplicar uma técnica de redução de dimensionalidade simples chamada <b>Singular Value Decomposition (SVD)</b>. 

<p>Essa técnica transformará os vetores da matriz original, rotacionando e escalando-os, resultando em novas representações. A redução de dimensionalidade é feita ao manter apenas as <i>k</i> dimensões mais representativas que escolhermos. Outra vantagem dessa técnica é que as dimensões originais são, de certa forma, "combinadas", o que resulta em uma nova forma de representar a combinação de termos. No contexto de PLN, essa técnica é conhecida como <b>Latent Semantic Analysis (LSA)</b></p>

In [44]:
from sklearn.decomposition import TruncatedSVD

svd_transformer = TruncatedSVD(n_components=1000)

svd_transformer.fit(tfs)

print(sorted(svd_transformer.explained_variance_ratio_)[::-1][:30])

[0.043191439648642374, 0.018619648885290087, 0.015991826821457032, 0.015984996712032983, 0.013744333318751027, 0.011771580238263269, 0.0096502136679668584, 0.0095168074249577239, 0.0095050502041712642, 0.0094605420641270359, 0.0093174536752600786, 0.0092700425653034994, 0.0077114068911936767, 0.0069839976227543009, 0.0066546888854482691, 0.0064195285763153668, 0.0058853020300814228, 0.0053722779003109221, 0.0051581711906630396, 0.0049312579564586645, 0.0048675271120122597, 0.0047897360657407888, 0.0044624928648066275, 0.0043399578651388731, 0.0042969022434977055, 0.0041605617284246462, 0.0040394806462126986, 0.0039137333697909643, 0.0038585290000539532, 0.0038298284370160505]


<p>Agora vamos manter as dimensões até que a variância acumulada seja maior ou igual a 0.50.</p>

In [45]:
cummulative_variance = 0.0
k = 0
for var in sorted(svd_transformer.explained_variance_ratio_)[::-1]:
    cummulative_variance += var
    if cummulative_variance >= 0.5:
        break
    else:
        k += 1
        
print(k)

143


<p>Transformarmos novamente, mas desta vez com o número de k componentes que obtemos anteriormente.</p>

In [48]:
svd_transformer = TruncatedSVD(n_components=k)
svd_data = svd_transformer.fit_transform(tfs)
print(sorted(svd_transformer.explained_variance_ratio_)[::-1])

[0.043191439648826289, 0.01861964888652147, 0.015991826835527356, 0.015984996710452587, 0.013744333334787466, 0.011771580244008924, 0.0096502136852790223, 0.009516807505667994, 0.0095050503834325385, 0.0094605423989449471, 0.0093174536840222762, 0.009270042723600538, 0.0077114073827614536, 0.0069839970437553236, 0.006654689038930575, 0.0064195307095592548, 0.0058853027737397496, 0.0053722773814358629, 0.0051581709690178513, 0.0049312580891063152, 0.0048675262334294039, 0.0047897364422535612, 0.0044625005241406192, 0.0043399605163811986, 0.0042968981982684333, 0.0041605573117342188, 0.0040394781087619638, 0.003913724919905005, 0.0038585228295508495, 0.0038298238387394429, 0.0037136224146103636, 0.003663400117768618, 0.0035824603355119099, 0.0035617267544038435, 0.0035351536998920238, 0.0034284434150083074, 0.0033450623521941829, 0.0032429636522826938, 0.003234138650533474, 0.0031762289053079997, 0.0031226961994305809, 0.0030422387153360387, 0.0029313662750773835, 0.0029182937754962499, 

In [50]:
print(svd_data.shape)

(2355, 143)
